In [59]:
import pandas as pd
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression, ElasticNet, LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, plot_tree, DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.compose import *
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier, VotingRegressor, BaggingClassifier
from sklearn.ensemble import *
from tqdm import tqdm
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

### AdaBoost

In [ ]:

df = pd.read_csv(r"D:\Cases\Sonar\Sonar.csv")
df
x, y = df.drop('Class', axis=1), df['Class']

le = LabelEncoder()
y = le.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)
dtc = DecisionTreeClassifier(random_state=42, max_depth=1)
nb = GaussianNB()
lr = LogisticRegression()
classifiers = [dtc, nb, lr]
n_estimators = [5, 10, 20, 50, 100, 150, 200, 400]
scores = []
for c in classifiers:
    for i in n_estimators:
        model = AdaBoostClassifier(estimator=c, n_estimators=i, random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)
        scores.append([c, i, log_loss(y_test, y_pred)])

df_scores = pd.DataFrame(scores, columns=['classifiers', 'n_estimators', 'log_loss'])
df_scores.sort_values(by='log_loss')
dtc = DecisionTreeClassifier(max_depth=1, random_state=42)  # Weak learner
lr = LogisticRegression(solver='liblinear', random_state=42)
nb = GaussianNB()

parm_grid = [
    [DecisionTreeClassifier(random_state=42),
     {'max_depth': [1, 2, 5, 10, 12, 15, 20],
      'min_samples_split': [5, 10, 15, 20],
      'min_samples_leaf': [2, 5, 8, 10, 12, 15]}],

    [GaussianNB(), {}],

    [LogisticRegression(solver='liblinear', max_iter=1000, random_state=42),
     {"penalty": ['l1', 'l2'],
      "C": [0.01, 0.1, 1.0]}
     ],
    [RandomForestClassifier(),
     {"n_estimators": [50, 100, 150, 200, 250, 300, 500],
      "criterion": ["gini"],
      "max_depth": [1, 2, 5, 10, 12, 15, 20],
      "min_samples_split": [5, 10, 15, 20],
      "min_samples_leaf": [2, 5, 8, 10, 12, 15]}
     ],

    [AdaBoostClassifier(),
     {"estimator": [dtc, nb, lr],
      "n_estimators": [5, 10, 20, 50, 100, 150, 200, 400]}
     ]

]
scores = []
for model, params in parm_grid:
    grid = GridSearchCV(estimator=model, param_grid=params, cv=5, n_jobs=-1, scoring='accuracy')
    grid.fit(X_train, y_train)
    scores.append({'model': model, "best_params": grid.best_params_, 'best_score': grid.best_score_})

df_scores = pd.DataFrame(scores)

### Gradient Boosting

In [2]:
df = pd.read_csv(r"D:\Cases\Sonar\Sonar.csv")
df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,0.0187,0.0346,0.0168,0.0177,0.0393,0.1630,0.2028,0.1694,0.2328,0.2684,...,0.0116,0.0098,0.0199,0.0033,0.0101,0.0065,0.0115,0.0193,0.0157,M
204,0.0323,0.0101,0.0298,0.0564,0.0760,0.0958,0.0990,0.1018,0.1030,0.2154,...,0.0061,0.0093,0.0135,0.0063,0.0063,0.0034,0.0032,0.0062,0.0067,M
205,0.0522,0.0437,0.0180,0.0292,0.0351,0.1171,0.1257,0.1178,0.1258,0.2529,...,0.0160,0.0029,0.0051,0.0062,0.0089,0.0140,0.0138,0.0077,0.0031,M
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048,M


In [3]:
df.isna().sum()

V1       0
V2       0
V3       0
V4       0
V5       0
        ..
V57      0
V58      0
V59      0
V60      0
Class    0
Length: 61, dtype: int64

In [6]:
X, y = df.drop('Class', axis=1), df['Class']

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.30,random_state = 25)

In [10]:
le = LabelEncoder()
le.fit_transform(X_test)
le.fit(y_test)

ValueError: y should be a 1d array, got an array of shape (63, 60) instead.

In [56]:

rates = np.linspace(0.01,0.8,20)
n_est = [50,100,200]
depths = [3,5,None]
scores = []
for r in tqdm(rates):
    for n in n_est:
        for d in depths:
            gbm = GradientBoostingClassifier(random_state = 25,n_estimators = n,max_depth =d,learning_rate = r)
            gbm.fit(X_train,y_train)
            y_pred_prob = gbm.predict_proba(X_test)
            scores.append([r,n,d,log_loss(y_test,y_pred_prob)])
df_scores = pd.DataFrame(scores,columns = ['rate','n_est','depth','score'])
df_scores.sort_values('score',ascending = False)

100%|██████████| 20/20 [01:13<00:00,  3.67s/it]


,rate,n_est,depth,score
110,0.508947,50,NaN,4.076914
113,0.508947,100,NaN,4.076914
116,0.508947,200,NaN,4.076914
161,0.716842,200,NaN,4.071150
158,0.716842,100,NaN,4.071150
...,...,...,...,...
27,0.134737,50,3.0,0.481256
12,0.051579,100,3.0,0.476514
6,0.010000,200,3.0,0.472457
9,0.051579,50,3.0,0.461801


### Light GBM

In [49]:
!pip install lightgbm

   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---------------------------------------- 1.5/1.5 MB 12.4 MB/s  0:00:00



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
rates = np.linspace(0.01,0.8,20)
n_est = [50,100,200]
# n_est = [i for i in range()]
depths = [3,5,None]
scores = []
for r in tqdm(rates):
    for n in n_est:
        for d in depths:
            gbm = LGBMClassifier(verbose  =-1 ,random_state = 25,n_estimators = n,max_depth =d,learning_rate = r)
            gbm.fit(X_train,y_train)
            y_pred_prob = gbm.predict_proba(X_test)
            scores.append([r,n,d,log_loss(y_test,y_pred_prob)])
df_scores = pd.DataFrame(scores,columns = ['rate','n_est','depth','score'])
df_scores.sort_values('score',ascending = False)

100%|██████████| 20/20 [00:04<00:00,  4.27it/s]


,rate,n_est,depth,score
152,0.675263,200,NaN,0.850884
151,0.675263,200,5.0,0.850884
149,0.675263,100,NaN,0.850884
148,0.675263,100,5.0,0.850884
146,0.675263,50,NaN,0.831909
...,...,...,...,...
37,0.176316,50,5.0,0.339264
27,0.134737,50,3.0,0.333093
21,0.093158,100,3.0,0.330297
46,0.217895,50,5.0,0.328558


### CatBoost

In [57]:
!pip install catboost

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ------ --------------------------------- 16.3/102.4 MB 85.3 MB/s eta 0:00:02
   ------------- -------------------------- 34.3/102.4 MB 84.4 MB/s eta 0:00:01
   -------------------- ------------------- 51.4/102.4 MB 82.7 MB/s eta 0:00:01
   ------------------------- -------------- 65.3/102.4 MB 78.1 MB/s eta 0:00:01
   ------------------------------- -------- 80.0/102.4 MB 76.2 MB/s eta 0:00:01
   ---------------------------------- ----- 89.4/102.4 MB 70.5 MB/s eta 0:00:01
   -------------------------------------- - 97.8/102.4 MB 66.0 MB/s eta 0:00:01
   --------------------------------------  102.2/102.4 MB 64.6 MB/s eta 0:00:01
   --------------------------------------  102.2/102.4 MB 64.6 MB/s eta 0:00:01
   ---------------------------------------- 102.4/102.4 MB 49.9 MB/s  0:00:02
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ------------------------------------- -- 9.2/9.8 MB 45.7 MB


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [60]:
rates = np.linspace(0.01,0.8,20)
n_est = [50,100,200]
# n_est = [i for i in range()]
depths = [3,5,None]
scores = []
for r in tqdm(rates):
    for n in n_est:
        for d in depths:
            gbm = CatBoostClassifier(random_state = 25,n_estimators = n,max_depth =d,learning_rate = r)
            gbm.fit(X_train,y_train)
            y_pred_prob = gbm.predict_proba(X_test)
            scores.append([r,n,d,log_loss(y_test,y_pred_prob)])
df_scores = pd.DataFrame(scores,columns = ['rate','n_est','depth','score'])
df_scores.sort_values('score',ascending = False)

  0%|          | 0/20 [00:00<?, ?it/s]

0:	learn: 0.6886437	total: 142ms	remaining: 6.95s
1:	learn: 0.6837603	total: 145ms	remaining: 3.48s
2:	learn: 0.6805421	total: 148ms	remaining: 2.31s
3:	learn: 0.6750598	total: 153ms	remaining: 1.76s
4:	learn: 0.6705323	total: 156ms	remaining: 1.4s
5:	learn: 0.6642440	total: 158ms	remaining: 1.16s
6:	learn: 0.6583878	total: 161ms	remaining: 989ms
7:	learn: 0.6543042	total: 164ms	remaining: 862ms
8:	learn: 0.6507337	total: 168ms	remaining: 765ms
9:	learn: 0.6427713	total: 171ms	remaining: 683ms
10:	learn: 0.6393470	total: 173ms	remaining: 613ms
11:	learn: 0.6343973	total: 175ms	remaining: 554ms
12:	learn: 0.6298419	total: 177ms	remaining: 504ms
13:	learn: 0.6265479	total: 180ms	remaining: 462ms
14:	learn: 0.6234433	total: 182ms	remaining: 424ms
15:	learn: 0.6193112	total: 184ms	remaining: 390ms
16:	learn: 0.6161448	total: 186ms	remaining: 360ms
17:	learn: 0.6097224	total: 188ms	remaining: 333ms
18:	learn: 0.6055790	total: 190ms	remaining: 309ms
19:	learn: 0.6034234	total: 192ms	remainin

  5%|▌         | 1/20 [00:06<01:59,  6.30s/it]

0:	learn: 0.6705277	total: 6.69ms	remaining: 328ms
1:	learn: 0.6472138	total: 12.8ms	remaining: 308ms
2:	learn: 0.6308514	total: 18.1ms	remaining: 284ms
3:	learn: 0.6068250	total: 21.7ms	remaining: 249ms
4:	learn: 0.5901443	total: 25.5ms	remaining: 229ms
5:	learn: 0.5801700	total: 28.5ms	remaining: 209ms
6:	learn: 0.5636830	total: 31.2ms	remaining: 191ms
7:	learn: 0.5490920	total: 33.6ms	remaining: 177ms
8:	learn: 0.5362981	total: 36ms	remaining: 164ms
9:	learn: 0.5188792	total: 38.2ms	remaining: 153ms
10:	learn: 0.5086511	total: 40.2ms	remaining: 143ms
11:	learn: 0.4849769	total: 42.3ms	remaining: 134ms
12:	learn: 0.4740489	total: 44.7ms	remaining: 127ms
13:	learn: 0.4656098	total: 47ms	remaining: 121ms
14:	learn: 0.4576295	total: 49.4ms	remaining: 115ms
15:	learn: 0.4414352	total: 51.5ms	remaining: 109ms
16:	learn: 0.4341935	total: 53.7ms	remaining: 104ms
17:	learn: 0.4168269	total: 56.1ms	remaining: 99.8ms
18:	learn: 0.4104909	total: 58.6ms	remaining: 95.5ms
19:	learn: 0.4046628	tot

 10%|█         | 2/20 [00:11<01:46,  5.94s/it]

...

0:	learn: 0.5027600	total: 4.48ms	remaining: 220ms
1:	learn: 0.4068234	total: 7.79ms	remaining: 187ms
2:	learn: 0.3603428	total: 11.1ms	remaining: 175ms
3:	learn: 0.3132400	total: 14.4ms	remaining: 166ms
4:	learn: 0.2541465	total: 17.1ms	remaining: 154ms
5:	learn: 0.2104570	total: 19.8ms	remaining: 145ms
6:	learn: 0.1728088	total: 22.4ms	remaining: 137ms
7:	learn: 0.1351174	total: 24.7ms	remaining: 130ms
8:	learn: 0.1185432	total: 27.2ms	remaining: 124ms
9:	learn: 0.0832036	total: 29.6ms	remaining: 118ms
10:	learn: 0.0604011	total: 31.8ms	remaining: 113ms
11:	learn: 0.0514367	total: 34ms	remaining: 108ms
12:	learn: 0.0413771	total: 36.4ms	remaining: 104ms
13:	learn: 0.0394623	total: 38.7ms	remaining: 99.4ms
14:	learn: 0.0359534	total: 41ms	remaining: 95.6ms
15:	learn: 0.0300680	total: 43.3ms	remaining: 92.1ms
16:	learn: 0.0254486	total: 45.7ms	remaining: 88.8ms
17:	learn: 0.0229124	total: 48ms	remaining: 85.3ms
18:	learn: 0.0205999	total: 50.2ms	remaining: 82ms
19:	learn: 0.0183009	tot

 95%|█████████▌| 19/20 [01:49<00:05,  5.77s/it]

0:	learn: 0.4997075	total: 7.94ms	remaining: 389ms
1:	learn: 0.4031230	total: 14.7ms	remaining: 353ms
2:	learn: 0.3541122	total: 19.7ms	remaining: 309ms
3:	learn: 0.3026035	total: 24.8ms	remaining: 285ms
4:	learn: 0.2394962	total: 28.1ms	remaining: 253ms
5:	learn: 0.1540320	total: 31.5ms	remaining: 231ms
6:	learn: 0.1133683	total: 34.3ms	remaining: 211ms
7:	learn: 0.0996992	total: 36.7ms	remaining: 193ms
8:	learn: 0.0905527	total: 39.1ms	remaining: 178ms
9:	learn: 0.0679504	total: 41.5ms	remaining: 166ms
10:	learn: 0.0609052	total: 43.9ms	remaining: 156ms
11:	learn: 0.0432161	total: 46.5ms	remaining: 147ms
12:	learn: 0.0404947	total: 48.9ms	remaining: 139ms
13:	learn: 0.0392827	total: 51.2ms	remaining: 132ms
14:	learn: 0.0316974	total: 53.4ms	remaining: 125ms
15:	learn: 0.0290977	total: 56ms	remaining: 119ms
16:	learn: 0.0266206	total: 58.5ms	remaining: 114ms
17:	learn: 0.0216343	total: 61ms	remaining: 108ms
18:	learn: 0.0172605	total: 63.7ms	remaining: 104ms
19:	learn: 0.0151040	total

100%|██████████| 20/20 [01:54<00:00,  5.75s/it]


,rate,n_est,depth,score
178,0.800000,200,5.0,0.668379
172,0.800000,50,5.0,0.661367
175,0.800000,100,5.0,0.660814
115,0.508947,200,5.0,0.635276
112,0.508947,100,5.0,0.629493
...,...,...,...,...
23,0.093158,100,NaN,0.349552
47,0.217895,50,NaN,0.346252
41,0.176316,100,NaN,0.343358
17,0.051579,200,NaN,0.339070


### application to housig

In [94]:
data = pd.read_csv(r"D:\Datasets\Housing.csv")

In [95]:
data

,price,lotsize,bedrooms,bathrms,stories,driveway,recroom,fullbase,gashw,airco,garagepl,prefarea
0,42000.0,5850,3,1,2,yes,no,yes,no,no,1,no
1,38500.0,4000,2,1,1,yes,no,no,no,no,0,no
2,49500.0,3060,3,1,1,yes,no,no,no,no,0,no
3,60500.0,6650,3,1,2,yes,yes,no,no,no,0,no
4,61000.0,6360,2,1,1,yes,no,no,no,no,0,no
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,yes,yes,no,no,yes,0,no
542,94000.0,6000,3,2,4,yes,no,no,no,yes,0,no
543,103000.0,6000,3,2,4,yes,yes,no,no,yes,1,no
544,105000.0,6000,3,2,2,yes,yes,no,no,yes,1,no


In [96]:
data.isna().sum()

price       0
lotsize     0
bedrooms    0
bathrms     0
stories     0
driveway    0
recroom     0
fullbase    0
gashw       0
airco       0
garagepl    0
prefarea    0
dtype: int64

In [97]:
data = pd.get_dummies(data,sparse = False,drop_first = True)
data

,price,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,42000.0,5850,3,1,2,1,True,False,True,False,False,False
1,38500.0,4000,2,1,1,0,True,False,False,False,False,False
2,49500.0,3060,3,1,1,0,True,False,False,False,False,False
3,60500.0,6650,3,1,2,0,True,True,False,False,False,False
4,61000.0,6360,2,1,1,0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
541,91500.0,4800,3,2,4,0,True,True,False,False,True,False
542,94000.0,6000,3,2,4,0,True,False,False,False,True,False
543,103000.0,6000,3,2,4,1,True,True,False,False,True,False
544,105000.0,6000,3,2,2,1,True,True,False,False,True,False


In [98]:
X = data.drop('price',axis = 1)
y = data['price']

In [99]:
X

,lotsize,bedrooms,bathrms,stories,garagepl,driveway_yes,recroom_yes,fullbase_yes,gashw_yes,airco_yes,prefarea_yes
0,5850,3,1,2,1,True,False,True,False,False,False
1,4000,2,1,1,0,True,False,False,False,False,False
2,3060,3,1,1,0,True,False,False,False,False,False
3,6650,3,1,2,0,True,True,False,False,False,False
4,6360,2,1,1,0,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
541,4800,3,2,4,0,True,True,False,False,True,False
542,6000,3,2,4,0,True,False,False,False,True,False
543,6000,3,2,4,1,True,True,False,False,True,False
544,6000,3,2,2,1,True,True,False,False,True,False


In [100]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 25)

In [101]:
sc = StandardScaler().set_output(transform = 'pandas')
X_train = sc.fit_transform(X_train)
y_train = sc.transform(X_train)